In [15]:
import numpy as np
import scipy
from imblearn.under_sampling import RandomUnderSampler
from models import AnonymousColorDetector
from utils import read_labeled_img

In [16]:
train_from_existed = False # 是否从现有数据训练，如果是的话，那就从dataset_file训练，否则就用data_dir里头的数据
data_dir = "data/dataset" # 数据集，文件夹下必须包含`img`和`label`两个文件夹，放置相同文件名的图片和label
dataset_file = "data/dataset/dataset_2022-07-20_10-04.mat"

# color_dict = {(0, 0, 255): "yangeng", (255, 0, 0): 'beijing',(0, 255, 0): "zibian"} # 颜色对应的类别
color_dict = {(0, 0, 255): "yangeng"}
# color_dict = {(255, 0, 0): 'beijing'}
# color_dict = {(0, 255, 0): "zibian"}
label_index = {"yangeng": 1, "beijing": 0, "zibian":2} # 类别对应的序号
show_samples = False # 是否展示样本

# 定义一些训练量
threshold = 2  # 正样本周围多大范围内的还算是正样本
node_num = 20  # 如果使用ELM作为分类器物，有多少的节点
negative_sample_num = None  # None或者一个数字，对应生成的负样本数量

## 读取数据

In [17]:
dataset = read_labeled_img(data_dir, color_dict=color_dict, is_ps_color_space=False)
if show_samples:
    from utils import lab_scatter
    lab_scatter(dataset, class_max_num=30000, is_3d=True, is_ps_color_space=False)

## 数据平衡化

In [18]:
if len(dataset) > 1:
    rus = RandomUnderSampler(random_state=0)
    x_list, y_list = np.concatenate([v for k, v in dataset.items()], axis=0).tolist(), \
                     np.concatenate([np.ones((v.shape[0],)) * label_index[k] for k, v in dataset.items()], axis=0).tolist()
    x_resampled, y_resampled = rus.fit_resample(x_list, y_list)
    dataset = {"inside": np.array(x_resampled)}

## 模型训练

In [19]:
# 对数据进行预处理
x = np.concatenate([v for k, v in dataset.items()], axis=0)
negative_sample_num = int(x.shape[0] * 1.2) if negative_sample_num is None else negative_sample_num
model = AnonymousColorDetector()

In [20]:
if train_from_existed:
    data = scipy.io.loadmat(dataset_file)
    x, y = data['x'], data['y'].ravel()
    model.fit(x, y=y, is_generate_negative=False, model_selection='dt')
else:
    world_boundary = np.array([0, 0, 0, 255, 255, 255])
    model.fit(x, world_boundary, threshold, negative_sample_size=negative_sample_num, train_size=0.7,
          is_save_dataset=True, model_selection='dt')
model.save()


 97%|███████████████████████████████████████████████████████▌ | 8785/9019 [00:07<00:00, 1208.04it/s]
9020it [00:07, 1241.41it/s]                                                                         

              precision    recall  f1-score   support

           0       1.00      1.00      1.00      2706
           1       1.00      1.00      1.00      2255

    accuracy                           1.00      4961
   macro avg       1.00      1.00      1.00      4961
weighted avg       1.00      1.00      1.00      4961



## 模型的可视化

In [21]:
model_path = "dt_2022-07-21_17-19.model"
dataset_path = "dataset_2022-07-21_17-19.mat"

In [22]:
data = scipy.io.loadmat(dataset_path)
ground_truth = data["x"][data["y"].ravel()==1]

FileNotFoundError: [Errno 2] No such file or directory: 'dataset_2022-07-21_17-19.mat'

In [ ]:
%matplotlib notebook
model = AnonymousColorDetector(model_path)
model.visualize(world_boundary = np.array([0, 0, 0, 255, 255, 255]),sample_size=5000,ground_truth=ground_truth)